<h2>Unit 12 Live Session </center> </h2>
<h4> W203 Instructional Team </h4>

#### Multivariate Linear Regression, Inference
<center>![title](hypothesis_1.jpg)</center>

#### Class Announcements
1. Announcement 1
2. Announcement 2
3. Announcement 3

#### 1.0 Useful functions in R:

Code    |    Function 
--------------------------|---------------------------------------------------------------------
coefficients(fit) | Extract model coefficients
fitted(fit)       | Extract predicted values
residuals(fit)    | Extract resduals
vcovHC(fit)       | Extract heteroskedasticity-robust covariance matrix
coeftest(fit, vcov = vcovHC) | Conduct hypothesis test with heteroskedasticity-robust standard errors
confint(fit, level=0.95)| Calculate non-robust CIs for model parameters (at 95%) 

**Note:** For heteroskedasticity-robust confidence intervals, get the variance of each coefficient from vcovHC, take the square root to get the standard error, get the proper t critical values from qt, and construct manually.

#### 2.0 Variance of OLS Estimators
Recall (one of) the expression(s) for the variance of each OLS slope coefficient:

$$var(\hat{\beta_j}) = \frac{\sigma^2}{SST_j (1-R_j^2)}$$

** A Crappy Analogy: ** Sometimes when you are (re)learning linear regression it is helpful to think in terms of analogies, so here goes... 

Imagine that you have gone to popular club late on a friday night with two friends Alex and Bob. You find yourself a table and sit down close to the dance floor. This would be the perfect opportunity for you to try out the new shopping cart dance move you have been practicing in front of your dog all week but its been a long night and the three you only want to talk.

Your conversation starts but you are having trouble following the conversation for two distinct reasons. 

1. You are having trouble distiguishiing the voices of your friends from the overall noise in the club. 

2. The lights are flashing in such a way that the faces of your friends are in total darkness most of the time, as a result you even when you can distinguish between the background club noises and your friends you ar having trouble distinguishing Alex's voice from Bob's. 

The background noise in the club is analagous to the error term $u$ of the regression,

$$ Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + u $$

$u$ is the variation in your outcome $Y$ (sound in the club) which is just uninteresting noise. 

In this context regression is an attempt to distinguish the variation in $Y$ (sound in the club) due to regressors $X_1$ and $X_2$, (Alex and Bob respectively) from the noise $u$ (Boots and Pants, "it's getting hot in here so ....", " Yeah, yeah, ... ", " oh my good she is such a ...."). 

Regression is also an attempt to distinguish the variation in $Y$ (sound in the club) due to $X_1$ (Alex) from the variation in $Y$ due to $X_2$ (Bob).

** 2.1 ** Why is it desirable to have a small variance for each estimated coefficient? 

> The practical consequences of a small variance is that it makes it easier to reject the null hypothesis.

> Suppose we estimate $\hat{\beta}_1 =-0.56$ with $n = 50$ and we would like to conduct the following test $H_0: \beta_1 = 0$ v.s. $H_1: \beta_1 \neq 0$ where in Case 1: $var(\hat{\beta}_1) = 0.25$ and in Case 2: $var(\hat{\beta}_1) = 0.35$ 

> We will calculate the p-value for each case and compare the results.

In [8]:
beta_hat = -0.56
v1 = 0.25
v2 = 0.35
p1 = 2*pt(beta_hat/v1,49)
p2 = 2*pt(beta_hat/v2,49)
paste( 'Case 1 p value: ', p1)
paste( 'Case 2 p value: ', p2)

[1] "Case 1 p value:  0.0296599572025266"

[1] "Case 2 p value:  0.116027262448521"

> If $\alpha = 0.5$ then we would have rejected $H_0$ in case 1 and failed ot reject $H_0$ is case 2.

** 2.2 **  For each component of this equation, explain (1) what it means, and (2) why it moves the standard error of $\beta_j$ up or down.

* $\sigma^2$
* $SST_j$
* $R_j^2$

> $\sigma^2$ is the variance of the error term of the regression, it makes sense that as the amplitude (or amount) of the "noise" in the regression increases the variance of the dependent variable increases (holding all independent variables constant), as a result the sampling distribution of $\hat{\beta}$ will increase since it depends directly on $\{Y_i\}_{i=1}^n$.    

> In our analogy think of $\sigma^2$ as the volume of the background noise in the club, as it gets louder you are less certain that it was either Alex or Bob who just put a bottle of crystal on your tab.

> $SST_j = \sum_{i=1}^n (X_i - \overline{X}_n)^2$ is the total sample variation in $X_i$. If we think of a regression as trying to distinguish the signal, the variation in $Y$ associated with $X$, from the noise, the variation in $Y$ associated with the error term the as the amplitude of the signal increases relative to the noise we should be able to estimate its effect on $Y$ with more precision.

> In our analogy think of $SST_j$ as the volume of one of your friends, say Alex. As he/she gets louder it is easier to distinguish them from the back ground noise. i.e. you are more certain that a slurred " I love you MAN!!!" came from Alex
rather than some random fist pumper (is that still a thing?).

> $R_j^2$ is the $R^2$ of a regression of $X_j$ on all other regressors it quantifies the closeness of the linear relationship between $X_j$ and all other regressors. In our analogy if the variation in $X_j$ is really closely related to the variation in $X_i$ it is more difficult to distiguish the relationship between $X_j$ and $Y$ from that between $X_i$ and $Y$ making estimates less precise.

> In our analogy think of $R_j^2$ as a measure of the similarities in the tone and vocal patterns of both Alex and Bob. As their voice become more similar, you are less certain whether it was Alex or Bob who just said, " Hold my cryssy, I am gonna go punch that bouncer !!!! ". 

Component 3 has a special name: the Variance Inflation Factor.  You can find the variance inflation factor for each variable in a linear model using the vif function in the car package.  Interpreting VIFs depends very much on context, but a VIF of 10 would usually be considered very high.

To get the variance of each coefficient in R, we would typically get the diagonal elements of the robust covariance matrix, diag(vcovHC(model))

To get the standard error of a coefficient, take the square root of the variance.

#### 3.0 R Exercise

In this analysis, we will use the mtcars dataset which is a dataset that was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973-74 models). The dataset is automatically available when you start R.  For more information about the dataset, use the R command: help(mtcars)

In [9]:
library(car)
library(lmtest)
library(sandwich)
library(stargazer)

ERROR: Error in library(car): there is no package called ‘car’


** 3.1 ** Using the mtcars data, run a multiple linear regression to find the effect of displacement (disp), gross horsepower (hp), weight (wt), and rear axle ratio (drat) on the miles per gallon (mpg).

In [10]:
#model <- lm(mpg~disp+hp+wt+drat, data=mtcars)

** 3.2: ** For ** each ** of the following 6 CLM assumptions, assess whether the assumption holds.  Where possible, demonstrate multiple ways of assessing an assumption.  When an assumption appears violated, state what steps you would take in response.

1. Linear population model
2. Random Sampling
3. No perfect multicollinearity
4. Zero-conditional mean
5. Homoskedasticity
6. Normality of Errors

> 1.) Linear population model: We don't have to check the linear population model, because we haven't constrained the error term, i.e haven't required it to be normal, so there's nothing to check at this point.

> 2.) Random Sampling: To check random sampling, we need background knowledge of how the data was collected.  Unfortunately, the description in R does not explain much about how the cars in the dataset were selected.

In [ ]:
# ?mtcars

> The data was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models).

> In general, we may be concerned about possible problems with independence.  For example, car makers might not want to create cars that are too similar to each other, to avoid cannibalizing sales.  They may pursue a more complicated strategy of designing a car for each of several market segments. 

> Finally, other car manufacturers may imitate particularly successful models.  For all these reasons, knowing the data on one car may inform the data we expect for another car.  If there are a large number of car manufacturers, we may expect these clustering (and also anti-clustering) effects to be small.

> Poorly constructed design, measurement scheme or limited range can be indicators that the
data was not randomly sampled.

> 3.) No perfect collinearity: No need to explicitly check for perfect collinearity, because R will alert us if this rare condition happens.